In [1]:
import sys
sys.path.append('..')
wp = '/home/jose/lung_cancer_ds_bowl/'

In [5]:
import os
import random
import logging
import multiprocessing
import numpy as np
import pandas as pd
from time import time
import matplotlib.pyplot as plt
from utils import plotting
from dl_model_pos_patches import  common
from sklearn import metrics
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback
from dl_networks.sample_resnet import ResnetBuilder
from dl_utils.tb_callback import TensorBoard




# PATHS
wp = os.environ['LUNG_PATH']


INPUT_PATH = '/mnt/hd2/preprocessed5'  # INPUT_PATH = wp + 'data/preprocessed5_sample'
VALIDATION_PATH = '/mnt/hd2/preprocessed5_validation_luna'
NODULES_PATH = wp + 'data/luna/annotations.csv'
#PATCHES_PATH = '/mnt/hd2/patches'  # PATCHES_PATH = wp + 'data/preprocessed5_patches'
PATCHES_PATH = '/home/jose/patches_temp'

OUTPUT_MODEL = wp + 'models/jm_patches_train_v17.hdf5'  # OUTPUT_MODEL = wp + 'personal/jm_patches_train_v06_local.hdf5'
LOGS_PATH = wp + 'logs/%s' % str('v17')

#LOGS_PATH = wp + 'logs/%s' % str(int(time()))
if not os.path.exists(LOGS_PATH):
    os.makedirs(LOGS_PATH)



# OTHER INITIALIZATIONS: tensorboard, model checkpoint and logging
#tb = TensorBoard(log_dir=LOGS_PATH, histogram_freq=1, write_graph=False, write_images=False)  # replace keras.callbacks.TensorBoard
#model_checkpoint = ModelCheckpoint(OUTPUT_MODEL, monitor='loss', save_best_only=True)
K.set_image_dim_ordering('th')
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s  %(levelname)-8s %(message)s',
                    datefmt='%m-%d %H:%M:%S')


### PATCHES GENERATION -----------------------------------------------------------------

## PATIENTS FILE LIST
patients_with_annotations = pd.read_csv(NODULES_PATH)  # filter patients with no annotations to avoid having to read them
patients_with_annotations = list(set(patients_with_annotations['seriesuid']))
patients_with_annotations = ["luna_%s.npz" % p.split('.')[-1] for p in patients_with_annotations]

filenames = os.listdir(INPUT_PATH)
filenames = [g for g in filenames if g.startswith('luna_')]
filenames_train = [os.path.join(INPUT_PATH, fp) for fp in filenames if fp in patients_with_annotations]
filenames_test = [os.path.join(VALIDATION_PATH, fp) for fp in os.listdir(VALIDATION_PATH) if fp in patients_with_annotations]



SLICE_DOWNSAMPLE = 12
SLICE_LUNG_SHAPE = (18,18)

def __load_and_store(filename):
    patient_data = np.load(filename)['arr_0']
    X, y, rois, stats = common.load_patient_3d_atlas(patient_data, 
                                                                 discard_empty_nodules=True, output_rois=True, debug=True, thickness=1,
                                                                 slice_downsample = SLICE_DOWNSAMPLE, slice_lung_shape=SLICE_LUNG_SHAPE
                                                                )
    logging.info("Patient: %s, stats: %s" % (filename.split('/')[-1], stats))
    return X, y, stats

common.multiproc_crop_generator(filenames_train,
                                os.path.join(PATCHES_PATH,'x_train_dl_pos_0.npz'),
                                os.path.join(PATCHES_PATH,'y_train_dl_pos_0.npz'),
                                __load_and_store, store=True)

common.multiproc_crop_generator(filenames_test,
                                 os.path.join(PATCHES_PATH,'x_test_dl_pos_0.npz'),
                                 os.path.join(PATCHES_PATH,'y_test_dl_pos_0.npz'),
                                 __load_and_store)


04-02 14:18:38  INFO     Loading 0/5
04-02 14:18:45  INFO     ++ Slice 104, stats: {'fp': 25, 'tp': 1, 'fn': 0}
04-02 14:18:45  INFO     ++ Slice 105, stats: {'fp': 24, 'tp': 1, 'fn': 0}
04-02 14:18:45  INFO     ++ Slice 106, stats: {'fp': 27, 'tp': 1, 'fn': 0}
04-02 14:18:46  INFO     ++ Slice 107, stats: {'fp': 22, 'tp': 1, 'fn': 0}
04-02 14:18:46  INFO     ++ Slice 108, stats: {'fp': 30, 'tp': 1, 'fn': 0}
04-02 14:18:46  INFO     Patient: luna_271220641987745483198036913951.npz, stats: {'fp': 128, 'fn': 0, 'tp': 5}
04-02 14:18:46  INFO     Loading 1/5


(14, 18, 18)


04-02 14:18:51  INFO     ++ Slice 98, stats: {'fp': 31, 'tp': 1, 'fn': 0}
04-02 14:18:51  INFO     ++ Slice 99, stats: {'fp': 29, 'tp': 1, 'fn': 0}
04-02 14:18:51  INFO     ++ Slice 100, stats: {'fp': 33, 'tp': 1, 'fn': 0}
04-02 14:18:52  INFO     ++ Slice 138, stats: {'fp': 25, 'tp': 1, 'fn': 0}
04-02 14:18:52  INFO     ++ Slice 139, stats: {'fp': 25, 'tp': 1, 'fn': 0}
04-02 14:18:52  INFO     ++ Slice 140, stats: {'fp': 26, 'tp': 1, 'fn': 0}
04-02 14:18:52  INFO     Patient: luna_569096986145782511000054443951.npz, stats: {'fp': 169, 'fn': 0, 'tp': 6}
04-02 14:18:52  INFO     Loading 2/5


(14, 18, 18)


04-02 14:18:56  INFO     ++ Slice 66, stats: {'fp': 64, 'tp': 1, 'fn': 0}
04-02 14:18:56  INFO     ++ Slice 67, stats: {'fp': 61, 'tp': 1, 'fn': 0}
04-02 14:18:56  INFO     ++ Slice 68, stats: {'fp': 55, 'tp': 1, 'fn': 0}
04-02 14:18:57  INFO     ++ Slice 85, stats: {'fp': 35, 'tp': 1, 'fn': 0}
04-02 14:18:57  INFO     Patient: luna_242624386080831911167122628616.npz, stats: {'fp': 215, 'fn': 0, 'tp': 4}
04-02 14:18:57  INFO     Loading 3/5


...

04-02 14:19:17  INFO     ++ Slice 40, stats: {'fp': 20, 'tp': 1, 'fn': 0}
04-02 14:19:17  INFO     ++ Slice 41, stats: {'fp': 28, 'tp': 1, 'fn': 0}
04-02 14:19:17  INFO     ++ Slice 42, stats: {'fp': 26, 'tp': 1, 'fn': 0}
04-02 14:19:17  INFO     ++ Slice 43, stats: {'fp': 32, 'tp': 1, 'fn': 0}
04-02 14:19:17  INFO     ++ Slice 55, stats: {'fp': 51, 'tp': 1, 'fn': 0}
04-02 14:19:17  INFO     ++ Slice 56, stats: {'fp': 52, 'tp': 1, 'fn': 0}
04-02 14:19:18  INFO     ++ Slice 57, stats: {'fp': 50, 'tp': 1, 'fn': 0}
04-02 14:19:18  INFO     ++ Slice 58, stats: {'fp': 54, 'tp': 1, 'fn': 0}
04-02 14:19:18  INFO     ++ Slice 59, stats: {'fp': 55, 'tp': 1, 'fn': 0}
04-02 14:19:18  INFO     ++ Slice 81, stats: {'fp': 45, 'tp': 1, 'fn': 0}
04-02 14:19:18  INFO     ++ Slice 82, stats: {'fp': 41, 'tp': 1, 'fn': 0}
04-02 14:19:18  INFO     ++ Slice 83, stats: {'fp': 42, 'tp': 1, 'fn': 0}
04-02 14:19:19  INFO     Patient: luna_118140393257625250121502185026.npz, stats: {'fp': 496, 'fn': 0, 'tp': 12}

(14, 18, 18)


04-02 14:19:24  INFO     ++ Slice 158, stats: {'fp': 27, 'tp': 1, 'fn': 0}
04-02 14:19:24  INFO     ++ Slice 159, stats: {'fp': 20, 'tp': 1, 'fn': 0}
04-02 14:19:24  INFO     ++ Slice 160, stats: {'fp': 21, 'tp': 1, 'fn': 0}
04-02 14:19:24  INFO     Patient: luna_119515474430718803379832249911.npz, stats: {'fp': 68, 'fn': 0, 'tp': 3}
04-02 14:19:25  INFO     Total time: 20.00, total patients:71, stats: {'fp': 2049, 'tp': 60, 'fn': 0}


(15, 18, 18)


04-02 14:19:34  INFO     Finished saving files
